<a href="https://colab.research.google.com/github/Lindronics/honours_project/blob/master/notebooks/analysis/Animal_classifier_8_Augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Animal classifier

## Setup and imports



In [0]:
%tensorflow_version 2.x

In [4]:
!rm -rf /content/*
!wget --no-check-certificate -O animals_train_final.zip "https://onedrive.live.com/download?cid=5691066189D062AB&resid=5691066189D062AB%21230244&authkey=ALJGkjjp2SmxjW4"
!wget --no-check-certificate -O animals_test_final.zip "https://onedrive.live.com/download?cid=5691066189D062AB&resid=5691066189D062AB%21230243&authkey=ADV07tX3wXwmS7I"
print("=> Fetched data")
!unzip -q animals_train_final.zip
print("=> Unzipped train data")
!unzip -q animals_test_final.zip
print("=> Unzipped test data")

--2020-03-31 08:37:53--  https://onedrive.live.com/download?cid=5691066189D062AB&resid=5691066189D062AB%21230244&authkey=ALJGkjjp2SmxjW4
Resolving onedrive.live.com (onedrive.live.com)... 13.107.42.13
Connecting to onedrive.live.com (onedrive.live.com)|13.107.42.13|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://qr7pug.am.files.1drv.com/y4mX92nEoGzZ6FWkKf0VRbIFloXMnMj5eCibHtNKD0r0Jh01amNahqpyqYGALUgG-o5znNXG0QWB0oIA8bNii4oBB29CcRQ4YqVrDK6vLJmemI0tOMevS89_dbhLMWJOIeQGoZPy0Vm92ZrBCs7p8q19FiFVkSIQPDyvmpbF3Qxq6ggsPFYiPyG47dsPY0qaoRZz83nifizSuCoUG9tpxcr8Q/animals_train_final.zip?download&psid=1 [following]
--2020-03-31 08:37:53--  https://qr7pug.am.files.1drv.com/y4mX92nEoGzZ6FWkKf0VRbIFloXMnMj5eCibHtNKD0r0Jh01amNahqpyqYGALUgG-o5znNXG0QWB0oIA8bNii4oBB29CcRQ4YqVrDK6vLJmemI0tOMevS89_dbhLMWJOIeQGoZPy0Vm92ZrBCs7p8q19FiFVkSIQPDyvmpbF3Qxq6ggsPFYiPyG47dsPY0qaoRZz83nifizSuCoUG9tpxcr8Q/animals_train_final.zip?download&psid=1
Resolving qr7pug.am.files.1drv.com (

In [3]:
!rm -rf honours_project
!git clone https://github.com/Lindronics/honours_project
!cp honours_project/models/classification/trans_cfg.py .
!cp honours_project/preprocessing/generate_labels.py .

Cloning into 'honours_project'...
remote: Enumerating objects: 297, done.
remote: Counting objects: 100% (297/297), done.
remote: Compressing objects: 100% (227/227), done.
remote: Total 1721 (delta 160), reused 193 (delta 68), pack-reused 1424
Receiving objects: 100% (1721/1721), 76.28 MiB | 28.49 MiB/s, done.
Resolving deltas: 100% (930/930), done.


In [0]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow.keras as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns; sns.set()
import os
from os import listdir
from os.path import isdir, join
import shutil
from tqdm import tqdm
from datetime import datetime
import random

from itertools import count
from collections import defaultdict

from honours_project.models.classification.models import AbstractModel, ResNet
from honours_project.models.classification.dataset import Dataset

In [3]:
from honours_project.preprocessing.generate_labels import generate_labels, write_labels
from honours_project.preprocessing.augment import augment_dataset

filter_fn = lambda _: True

write_labels(generate_labels(filter_fn, "animals_test_final/images", channel_prefix=False), "animals_test_final/labels.txt")

train_labels = generate_labels(filter_fn, "animals_train_final/images", channel_prefix=False)
write_labels(train_labels, "animals_train_final/labels_1.txt")

write_labels(augment_dataset(train_labels, "animals_train_final/images_2", 2), "animals_train_final/labels_2.txt")
write_labels(augment_dataset(train_labels, "animals_train_final/images_5", 5), "animals_train_final/labels_5.txt")

  0%|          | 0/2396 [00:00<?, ?it/s]

Dataset composition:
 - silkie-chicken: 	81
 - hamburg-chicken: 	60
 - goose: 	100
 - chicken: 	119
 - cat: 	63
 - pony: 	61
 - muscovy-duck: 	64
 - alpaca: 	87
635 items total.
Dataset composition:
 - silkie-chicken: 	224
 - hamburg-chicken: 	338
 - goose: 	290
 - chicken: 	277
 - cat: 	266
 - pony: 	415
 - muscovy-duck: 	209
 - alpaca: 	377
2396 items total.


100%|██████████| 2396/2396 [03:31<00:00, 11.36it/s]


## Dataset

In [0]:
def train(train_labels, test_labels, epochs=80, register=False, batch_size=16):
    train_data = Dataset(train_labels, rgb_only=False, split=False, res=(120, 160), register=register, batch_size=batch_size)
    test_data = Dataset(test_labels, rgb_only=False, split=False, res=(120, 160), register=register, batch_size=batch_size)

    X, y = train_data.get_all()
    X_test, y_test = test_data.get_all()

    model = ResNet("fusion", train_data.num_classes(), train_data.shape(), None).get_model()
    optimizer = K.optimizers.Adam(learning_rate=0.000001, epsilon=0.005)
    # optimizer = K.optimizers.SGD(decay=1e-6)
    model.compile(optimizer=optimizer,
                loss="categorical_crossentropy",
                metrics=["accuracy"])

    tf.random.set_seed(42)
    np.random.seed(42)

    hist = model.fit(X, y, epochs=epochs, validation_data=(X_test, y_test), batch_size=batch_size)
    return model, hist.history

In [0]:
train_sets = [
    ("1", "animals_train_final/labels_1.txt"),
    ("2", "animals_train_final/labels_2.txt"),
    ("5", "animals_train_final/labels_5.txt"),
]

results = {}

for i, labels in train_sets:
    model, hist = train(labels, "animals_test_final/labels.txt", epochs=80, register=False, batch_size=32)
    results[i] = {"model": model, "hist": hist}

In [0]:
  for a in results:
    hist = results[a]["hist"]

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))

    ax1.plot(hist["loss"], label="train")
    ax1.plot(hist["val_loss"], label="val")
    ax1.set_title("Loss")

    ax2.plot(hist["accuracy"], label="train")
    ax2.plot(hist["val_accuracy"], label="val")
    ax2.set_title("Accuracy")
    fig.suptitle(a)

In [0]:
def analyse(name):
    test_data = Dataset("animals_test_final/labels.txt", rgb_only=False, split=False, res=(120, 160), register=False, batch_size=32)
    model = results[name]["model"]

    y_pred = np.argmax(model.predict(test_data), axis=1)
    y_test_ = test_data.get_labels()[:y_pred.shape[0]]
    print(classification_report(y_test_, y_pred, target_names=test_data.class_labels))

    # confusion
    y_pred_named = [test_data.class_labels[x] for x in y_pred]
    y_test_named = [test_data.class_labels[x] for x in y_test_]
    confusion = confusion_matrix(y_test_named, y_pred_named, labels=test_data.class_labels)

    f, ax = plt.subplots(figsize=(3, 3), dpi=150)
    ax = sns.heatmap(confusion, annot=True, xticklabels=test_data.class_labels, yticklabels=test_data.class_labels, square=True, cbar=False, fmt="d")
    ax.set(xlabel='Predicted class', ylabel='Actual class')
    f.savefig(f"confusion_{name}.pdf", bbox_inches="tight")
    plt.show()

In [0]:
for name in results:
    analyse(name)

In [0]:
import pickle
for name in results:
    with open(f"{name}.pickle", "wb") as f:
        pickle.dump(results[name]["hist"], f)